<a href="https://colab.research.google.com/github/alirezash97/Text-Classification/blob/master/NLP981_Phase1_checkpoint.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NLP981 Final Project - Phase #1

*   Instructor: Javad PourMostafa
*   Teaching Assistant: Parsa Abbasi
*   University of Guilan, 1st semester of 2019
*   GitHub repository : *https://github.com/JoyeBright/NLP*

In [0]:
!pip install nltk
!pip install stopwords_guilannlp
!pip install hazm

In [0]:
import nltk
from tqdm import tqdm
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [0]:
sentence = "Hello, world!"
print (nltk.word_tokenize(sentence))

['Hello', ',', 'world', '!']


It's the first phase of your final project for the *NLP981* course. The main idea behind this phase is to portray the develope side of *NLP*.

You must code inside of this python notebook. I highly recommend you to use the *Google Colab* environment. 

If you have any questions, feel free to ask.
You can use [*Quera*](https://quera.ir/course/4385/) platform for your general questions.



## Introduction

A category predictor is going to build at this phase of the project.

The predictor gets a text as input and predicts a category for that.

For this purpose, you need to :

1.   Load the dataset
2.   Preprocess the text data
3.   Implement a word representation method to represent each text as a numeric vector
4.   Implement a classification model and train that using the training set
5.   Predict a category for each of validation data using implemented model
6.   Measure your work using confusion matrix and some common metrics

**Important Note:** You can use any library you want in sections 1 and 2. But everything in section 3-6 need to be coded purely.



## 1) Dataset

The dataset you will use in this phase is called *Divar* that released by the *CafeBazaar* research team.

It contains more than 900,000 posts of the *Divar* ads platform. We split this dataset into training, validation, and testing sets.

The testing set is not accessible for you, and we use them to evaluate your work on the presentation day.

You can download the dataset files (training and validation sets) directly from the following link :

> *https://drive.google.com/open?id=1oj-fqpymjDr8QsOK-zQliiqXbVqakrFo*


### 1.1) Import

In [0]:
# Import the training and validation sets here
import stopwords_guilannlp as sw
from google.colab import files
from itertools import chain
import plotly as py
import plotly.graph_objs as go
import time
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import numpy as np
import matplotlib.pyplot as plt
import regex as re
import sys
from hazm import *
import pandas as pd
trainset = pd.read_csv('drive/My Drive/NLP_dataset/trainset.csv')
validationset = pd.read_csv('drive/My Drive/NLP_dataset/validationset.csv')

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### 1.2) Analyzing

Display the top 10 rows of the train set.

In [0]:
# Your code
trainset.head(10)

,Unnamed: 0,Unnamed: 0.1,archive_by_user,brand,cat1,cat2,cat3,city,created_at,desc,id,image_count,mileage,platform,price,title,type,year
0,0,282086,True,NaN,personal,clothing-and-shoes,shoes-belt-bag,Tehran,Monday 11AM,چکمه یکبار پوشیده شده قیمت 42\nکفش قهوه ای سوخ...,12875614029625,3,NaN,mobile,42000,سایز 40,women,NaN
1,1,762753,True,Samsung::سامسونگ,electronic-devices,mobile-tablet,mobile-phones,Tehran,Wednesday 12PM,گوشی رو تا حالا باز نکردم و تو جعبه پلمپه از د...,16051997226596,0,NaN,mobile,850000,گوشی سامسونگ a3 2016,NaN,NaN
2,2,805240,True,NaN,personal,jewelry-and-watches,watches,Tehran,Tuesday 09AM,ساعت هیچ مشکلی ندارد اصل اصل هستش چون دیگه دست...,51715717387979,2,NaN,mobile,130000,ساعت زنانه اسپریت اصل,NaN,NaN
3,3,556730,False,NaN,personal,baby-and-toys,personal-toys,Tehran,Thursday 06PM,دوچرخه از هرلحاظ سالمه و فقط مدت کوتاهی استفاد...,9381204619687,2,NaN,mobile,200000,فروش یک عدد دوچرخه مارک پرادو بسیار سالم,NaN,NaN
4,4,727332,True,NaN,leisure-hobbies,hobby-collectibles,coin-stamp,Tehran,Wednesday 09PM,14 اسکناس مطابق تصویر همه باهم 200 هزار تومان\...,47687757949429,3,NaN,web,-1,14 اسکناس کلکسیونی code1,NaN,NaN
5,5,805039,True,NaN,leisure-hobbies,sport-leisure,ball-sports,Tehran,Tuesday 11AM,فوتبال دستی حرفه ای تاشو درحد نو فقط 2ماه استف...,35888748102799,3,NaN,mobile,480000,فوتبال دستی حرفه ای,NaN,NaN
6,6,812617,False,NaN,for-the-home,furniture-and-home-decore,lighting,Tehran,Thursday 08AM,لوستر نو برای اتاق کودک,61553946516880,1,NaN,mobile,25000,لوستر,NaN,NaN
7,7,295730,True,NaN,vehicles,parts-accessories,NaN,Isfahan,Monday 10PM,4 عدد رینگ اهنی ال 90 سالم قیمت از شما من فروش...,21090520055338,0,NaN,mobile,-1,رینگ اهنی ال 90,NaN,NaN
8,8,777605,False,نیسان::Nissan,vehicles,cars,light,Shiraz,Wednesday 01PM,"بیمه تا۹۵/۹/۲۷,کف دستی رنگ درعقب",27046580911179,2,380000.0,mobile,40000000,نیسان سرانزا مدل۸۳,NaN,1383
9,9,797079,False,پراید صندوق‌دار::Pride,vehicles,cars,light,Tehran,Tuesday 03PM,عقب وجلوپلمب بیرنگ لاستیک نو به شرط مصرف کننده...,42106059496341,0,109000.0,mobile,14000000,پراید سفید,NaN,1390


How many data (rows) stored in the training and validation sets?

In [0]:
train_len = trainset.shape[0]
valid_len = validationset.shape[0]

How many posts are in each category (First level categories)? (Based on training set)

In [0]:
# Your code
print(trainset.groupby('cat1').size())

cat1
businesses             45660
electronic-devices    122905
for-the-home          214955
leisure-hobbies        61676
personal              102804
vehicles              152000
dtype: int64


## 2) Preprocessing

There are two kinds of text data in the dataset: *Title* and *Description*.
You can use one or both of them as text inputs of your classification model. Choose a composition that gives you a higher measuring score.

You need to apply some preprocessing procedures on your text data first. We want at least **4** preprocessing step from you. It can be removing stop words, removing punctation, removing or replacing digits, stemming, lemmatizing, normalization, and so on.

You can use the [*Stopwords Guilan NLP*](https://github.com/JoyeBright/stopwords_guilannlp) library to access a collection of Persian stop words.

In [0]:
def preprocessing(document):
  # Your preprocessing steps
    
    cleared_text = []
    normalizer = Normalizer()
    lemmatizer = Lemmatizer()
    stemmer = Stemmer()
    
    
    
    for text in tqdm(document) :
        
        lemmatized = []
        filtered_tokens = []
        
    # step 1 
        
        normalized_text = normalizer.normalize(text)
    
    # step 2
    
        tokens = nltk.word_tokenize(normalized_text)
        
    
    # step 3
    
        stopwords = sw.stopwords_output("Persian", "nar")
        for w in tokens: 
            if w not in stopwords :
                filtered_tokens.append(w)
                
    # step 4
        filtered_tokens = [token for token in filtered_tokens if not token.isdigit()]
   
    # step 5
        
        for i in range (0, len(filtered_tokens)):
            if len(lemmatizer.lemmatize(filtered_tokens[i])) >= 1:
                lemmatized.append(lemmatizer.lemmatize(filtered_tokens[i]))
            else :
                pass
    
        
        cleared_text.append(lemmatized)  


    return cleared_text

## 3) Word Representation

As you know, classification models can't deal with strings directly, and you have to represent your texts in a numerical form.

### 3.1) Tf-idf

You have to implement the tf-idf vectorization method from scratch in this step. 

Furthermore, a function must be implemented that gives a text input and return a tf-idf vectorized representation.

$$\text{tf-idf}(t, d) = \text{tf}(t, d) \times \text{idf}(t)$$

*tf* (term-frequency) is the count of occurrences of the word `t` in specific text `d`.

*idf* (inverse document-frequency) is term that is inversely proportional to the number of texts with the given word. It can be calculated this way:
$$\text{idf}(t) = \text{log}\frac{1 + n_d}{1 + n_{d(t)}} + 1$$
where $n_d$ is the whole number of texts and $n_{d(t)}$ is the number of texts with the word `t`.

In [0]:
##########################  train set for learning #######################
start_point = 0   # form #row 
end_point = 2000   # to   #row
################################################################################
H = trainset.iloc[start_point : end_point,9].values
document = preprocessing(H)

100%|██████████| 2000/2000 [00:06<00:00, 327.87it/s]


In [0]:
def tf(document):
    
    tf = []
    
    for text in tqdm(document):
        tf_text = {}
        for word in text:
            tf_text[word] = 0
        for word in text:
            tf_text[word] += 1
        for word in tf_text:
            # print(len(text))
            tf_text[word] = tf_text[word] / len(text)
        tf.append(tf_text)
    
                      
    return tf 

def idf(tf,word):
    nd = len(tf)
    ndt = 0
    idf_ = {}
    for text in tf :
        if word in text:
            ndt += 1
    idf = np.log( ((1 + nd) / (1 + ndt)) + 1)
    
    return idf



In [0]:

def tf_idf(document):
    tf_ = tf(document)
    idf_ = {}
   
    
    for text in tqdm(document):
        for word in text:
            idf_[word] = idf(tf_,word)
    
    tfidf = tf_
    for index, text in enumerate(tf_):
        for word in text:
            tfidf[index][word] = text[word] * idf_[word]
    
    vector = np.zeros((len(tf_), len(idf_)))
    
    for i, (k, v) in tqdm(enumerate(idf_.items())):
        temp = (k, v)
        for index, text in enumerate(tfidf):
            if k in text:
                vector[index][i] = text[k]
            else :
                
                vector[index][i] = 0
    
           
  # Your code
    return vector, idf_
tfidfmatrix, idf_train = tf_idf(document)

100%|██████████| 2000/2000 [00:02<00:00, 921.65it/s]
5581it [00:04, 1308.80it/s]


# 4) Classification

![alt text](https://cdn.lynda.com/course/578082/578082-637075371482276339-16x9.jpg)

### 4.1) Logistic Regression

The Logistic Regression classifier must be implemented from scratch here.

You can fit the training data into the classifier after implementing linear regression.

In [0]:
Y_train = trainset['cat1'].values.reshape((1, 700000))
Y_trainset = Y_train[0][start_point : end_point]
X = tfidfmatrix
def Y_vectorization(Y_trainset):
    vectorized_y = np.zeros((Y_trainset.shape[0], 6))
    for index, word in enumerate(Y_trainset) :
        if word == 'businesses':
            vectorized_y[index][0] = 1
        elif word == 'electronic-devices':
            vectorized_y[index][1] = 1
        elif word == 'for-the-home':
            vectorized_y[index][2] = 1
        elif word == 'leisure-hobbies':
            vectorized_y[index][3] = 1
        elif word == 'personal':
            vectorized_y[index][4] = 1
        elif word == 'vehicles':
            vectorized_y[index][5] = 1
    return vectorized_y
m = tfidfmatrix.shape[0]
number_of_features = tfidfmatrix.shape[1]


In [0]:
X = tfidfmatrix
Y = Y_vectorization(Y_trainset)
m = X.shape[0]
number_of_features = X.shape[1]
learning_rate = 0.05



w = np.zeros((number_of_features, 6))
Z = np.dot(X, w).reshape((m, 6)) 

def softmax(Z):
    Z = np.array(Z)
    output = np.zeros((Z.shape[0], Z.shape[1]))
    for index, row in enumerate(Z) :
        for index1, column in enumerate(row):
            output[index][index1] = np.exp(column) / np.sum(np.exp(row))
    return output

def gradient_descent(X, Z, Y):
    return (np.dot(X.T, (Z[0] - Y)) / len(Y)).reshape((number_of_features, 6))

def update_weight_loss(w, learning_rate, gradient):
    return w - learning_rate * gradient

for i in tqdm(range(0, 2000)) :
    Z = np.dot(X, w).reshape((m, 6))
    A = softmax(Z)
    gradient = gradient_descent(X, A, Y)
    w = update_weight_loss(w, learning_rate, gradient)



100%|██████████| 2000/2000 [07:07<00:00,  4.67it/s]


In [0]:
def final(A):
    output = np.zeros((A.shape[0], A.shape[1]))
    for index, row in enumerate(A):
        for index1, column in enumerate(row):
            if column == max(row):
                output[index][index1] = 1
    return output
        

In [0]:
W_for_unique_words = {}
for i, (k, v) in enumerate(idf_train.items()):
    W_for_unique_words[k] = w[i]

## 5) Prediction

Now you can predict a category for each of the validation data using the implemented classifier.

In [0]:
######################### validationset #####################
testset_start_point = 100000
testset_end_point = 100010
# a = validationset.iloc[testset_start_point : testset_end_point, 9].values
# print(a)
#############################################################

In [0]:
# Y_valid = validationset['cat1'].values.reshape((1, valid_len))
# Y_testset = Y_valid[0][testset_start_point : testset_end_point]
# Y_actual = Y_vectorization(Y_testset)
# A = validationset.iloc[testset_start_point : testset_end_point, 9].values
# print(A)
A = ['پلی استیشن 2درحدصفرو واقعا نونوهستش با دو دستگیره نونو دستگیره هارو تازه   صفر خریدم تو عکس مشخصه نونو هستش همه بازیهارو ب راحتی بالا میاره بهمراه تست خودتون یا علی',
'بیمه یکسال موتوری پلمپ ب شرط میدم اچار نخورده دزدگیر از راه دور جک سنسور دار   لاستیک جلو عقب نو لنت نو فقط باتریش خوابیده استفاده نکردم چندوقت باتریش ضعیف شده هیچ مشکل دیگه ی نداره',
'میز   و صندلی نهارخوری 4نفره چوبی, به رنگ  قهوه ای تیره با شیشه 10میل; جنس چوب مرغوب, سالم و تمیز',
' دور رنگ دوگانه دستی و شتاب اولیه هم تراز با بنزین بیمه بدنه دارد در صورت نخواستن کپسول سی ان جی برداشته میشود و میلغ ٢ میلیون تومان از کسر خواهد شد',
'بی رنگ بیمه تا برج 7',
'رنگ یشمی، عقب وجلو بدون تصادف، بیمه تااخرسال وشش سال تخفیف بیمه،  چهارحلقه لاستیک نو، باتری 88امپرنوباگارنتیش، رادیات دولول نو.موتورتازه.تعمیربه.شرط',
'شش جفت   قناری سفیدطلایی باقفس وکاورآوازخوان به فروش می رسدجفتی80000هزارتومان',
'گوشی هواوی وای 560 تمیز با جعبه و شارژر',
'محصول تیم خیریه ی خانه خلاقیت و هنر.ساخته شده از چوب نرّاد خشک روسی.گارانتی مادام العمر.ارسال رایگان.استحکام و ظرافت مثال زدنی.',
'باسلام..به سیستم در حد نو فقط یه دو ماه رو ماشین نصب بوده...با نو هیچ فرقی نمیکنه،،به شرط وتضمین،ضبط پایونر x1درحد نو مخصوص سیستم خیلی امکانات داره 4کاناله ',
'فقط روکش مبل آسیب دیده سه تا پایه مبل دیگه هیچ مشکلی نداره سالمه میز عسلی جلومبلی کاملا سالم نو .به خریدار واقعی تخفیف هم میدم چون جاندارم میخوام بفروشم',
'به دلیل اینکه  میخوام دکوراسیون خونمو عوض کنم میفروشمش به قیمت خیلی مناسب',
'رینگ ۱۸.۲۲۵.۶۵ یدونه ضعیف هستش.مشتری واقعی زنگ بزند ممنون.تالش',
'یک مکانی امن برای استراحت کردن، زندگی کردن و آسودن است در این مکان معمولاً یک فرد یا یک خانواده زندگی می‌کنند و وسایل خود را در آن نگهداری می‌کنند در فرهنگ سنتی و عامه ایرانی گاه از اصطلاح «چهاردیواری (چاردیواری)» به جای  استفاده می‌شود.[۱‌ جدید شامل سرویس‌های بهداشتی و امکانات تهیه غذا نیز هستند اما در برخی مناطق همچنان برخی اقوام دارای خانه‌هایی هستند که پیشرفت نکرده، از امکانات رفاهی یا حتی مکان ثابتی برخوردار نیست. چادرنشینی یکی از انواع این نوع زندگی است.']
B = np.array(A)
print(len(B))
document_test = preprocessing(A)
# print(document_test)

tf_test = tf(document_test)
vector_test, idf_test = tf_idf(document_test)
w_test = np.zeros((len(idf_test), 6))
for index, (word, value) in enumerate(idf_test.items()):             
     if word in W_for_unique_words:

        w_test[index]= W_for_unique_words[word]
     else:
         pass

Z_test = np.dot(vector_test, w_test)
A_test = softmax(Z_test)
Y_prediction = final(A_test)



14


100%|██████████| 14/14 [00:00<00:00, 8415.06it/s]
195it [00:00, 86003.08it/s]


In [0]:
print(Y_prediction)

[[0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]]


In [0]:
counter = 0
for index, row in enumerate(Y_actual):
    for index1, column in enumerate(row) :
        if column == 1:
            if Y_prediction[index][index1] == 1:
                counter += 1
            else :
                pass
        else :
            pass
print(counter)
                

1308


## 6) Evaluation

It's time to evaluate your model using predicted categories for validation data.

You need to create a confusion matrix based on your prediction and the real labels. Then you can use this confusion matrix for calculation other measuring metrics. 

As this problem is a multi-class problem, the calculation formula is a little different from the general case. Read [this article](https://towardsdatascience.com/multi-class-metrics-made-simple-part-i-precision-and-recall-9250280bddc2) for more information.

### 6.1) Confusion matrix

In [0]:
# Your implementation

confusion_matrix = np.zeros((6, 6))

for row_index_prediction, row in enumerate(Y_prediction):
    for column_index_prediction, column in enumerate(row):
        if column == 1:
            if Y_actual[row_index_prediction][column_index_prediction] == 1:
                confusion_matrix[column_index_prediction][column_index_prediction] += 1
            else:
                for column_index_actual, i in enumerate(Y_actual[row_index_prediction]):
                    if i == 1:
                        confusion_matrix[column_index_prediction][column_index_actual] +=1
                    else :
                        pass
        else:
            pass

print(confusion_matrix)
                    
    

[[  8.   9.  13.   6.   4.   6.]
 [ 17. 242.  54.  20.  27.  27.]
 [ 71.  77. 522.  57.  99.  48.]
 [  3.   8.  20.  75.   8.  12.]
 [  9.   9.  18.  11. 124.   7.]
 [ 23.  52.  69.  32.  36. 337.]]


### 6.2) Accuracy

$$\text{Accuracy} = \frac{TP + TN}{TP + FP + FN + TN}$$

In [0]:
# Your code
axis0 = np.sum(confusion_matrix, axis = 0)
accuracy = np.zeros((confusion_matrix.shape[0], 1))
for i in range(confusion_matrix.shape[0]):
  TP = confusion_matrix[i][i]
  FP = np.sum(confusion_matrix[i]) - confusion_matrix[i][i]
  FN = axis0[i] - confusion_matrix[i][i]
  TN = np.sum(confusion_matrix) - TP -FP -FN
  accuracy[i][0] = (TP + TN) / (TP + FN + FP + TN)
  print("accuracy for class ", cat1[i], " is : ", accuracy[i][0])
mean_accuracy = np.sum(accuracy/accuracy.shape[0])
print(mean_accuracy)


accuracy for class  businesses  is :  0.9254629629629629
accuracy for class  electronic-devices  is :  0.8611111111111112
accuracy for class  for-the-home  is :  0.7564814814814815
accuracy for class  leisure-hobbies  is :  0.9180555555555555
accuracy for class  personal  is :  0.8944444444444445
accuracy for class  vehicles  is :  0.8555555555555555
0.8685185185185185


### 6.3) Precision

$$\text{Precision} = \frac{TP}{TP + FP}$$

In [0]:
# Your code

cat1 = ["businesses", "electronic-devices", "for-the-home", "leisure-hobbies", "personal", "vehicles"]

precision = np.zeros((6, 1))
for i in range (0, 6):
    precision[i][0] = confusion_matrix[i][i] / np.sum(confusion_matrix[i])
    print("precision for class ", cat1[i], " is : ", precision[i][0])

precision for class  businesses  is :  0.17391304347826086
precision for class  electronic-devices  is :  0.6253229974160207
precision for class  for-the-home  is :  0.597254004576659
precision for class  leisure-hobbies  is :  0.5952380952380952
precision for class  personal  is :  0.6966292134831461
precision for class  vehicles  is :  0.6138433515482696


### 6.4) Recall

$$\text{Recall} = \frac{TP}{TP + FN}$$

In [0]:
# Your code
recall = np.zeros((6, 1))
axis0 = np.sum(confusion_matrix, axis = 0)
for i in range (0, 6):
    recall[i][0] = confusion_matrix[i][i] / axis0[i]
    print("recall for class ", cat1[i], " is : ", recall[i][0])

recall for class  businesses  is :  0.061068702290076333
recall for class  electronic-devices  is :  0.5941644562334217
recall for class  for-the-home  is :  0.8826219512195121
recall for class  leisure-hobbies  is :  0.3826530612244898
recall for class  personal  is :  0.4539249146757679
recall for class  vehicles  is :  0.7798594847775175


### 6.5) F1 score

$$\text{F1 score} = 2\times \frac{(Recall \times  Precision)}{Recall + Precision}$$ 

In [0]:
# Your code
f1_score = np.zeros((6, 1))
for i in range(0, 6):
    f1_score[i][0] = 2 * (recall[i][0] * precision[i][0]) / recall[i][0] + precision[i][0]
    print("F1 score for class ", cat1[i], " is : ", f1_score[i][0])


F1 score for class  businesses  is :  1.0
F1 score for class  electronic-devices  is :  2.3252595155709344
F1 score for class  for-the-home  is :  1.6782608695652173
F1 score for class  leisure-hobbies  is :  2.419354838709677
F1 score for class  personal  is :  2.375
F1 score for class  vehicles  is :  2.1210191082802545


## 7) K-Fold Cross Validation *(Optional)*

Evaluate your model based on the K-Fold Cross Validation approach. This step is optional and has a few extra points.

In [0]:
# Your implementation